# Import libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
from collections import Counter
import re

# Load the training csv's and combine them into one dataframe

In [2]:
# load all 8 training csv's
df1 = pd.read_csv('reviews/train-1.csv')
df2 = pd.read_csv('reviews/train-2.csv')
df3 = pd.read_csv('reviews/train-3.csv')
df4 = pd.read_csv('reviews/train-4.csv')
df5 = pd.read_csv('reviews/train-5.csv')
df6 = pd.read_csv('reviews/train-6.csv')
df7 = pd.read_csv('reviews/train-7.csv')
df8 = pd.read_csv('reviews/train-8.csv')

# combine the dataframes into one big dataframe
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])

In [3]:
def remove_non_ascii_2(text):

    return re.sub(r'[^\x00-\x7F]',' ', text)

In [4]:
df['review_body'] = df['review_body'].apply(lambda x: x.replace('<br />', ' '))
# df['review_body'] = df['review_body'].apply(remove_non_ascii_2)

# Show the head of the data

In [5]:
df.head()

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
0,9,B001N2MZT8,903886718,Green Zone [DVD],N,Y,green zone,I found at first it was a little difficult to ...,2010-11-15,1,3,False
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,NaN,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
2,19,1423165691,883799517,A Disney Sketchbook.,N,N,okay mais...,est-ce une coincidence que la plupart des prin...,2012-12-22,0,0,False
3,33,0061091480,623343977,Your Erroneous Zones,N,N,Arrogant,Wáyné Dyér is á pớpúlár áméricán pérsớnál grớw...,2009-07-21,0,0,True
4,34,B00HZ4CYOY,647510225,König der Mathematik Junior,N,Y,Tớllé Máthé Ápp...,.....unsere Kids mögen diese Art des Lernens. ...,2015-06-01,0,1,False


# Check if the unnamed column could be the review id

In [6]:
# print the number of rows and number of unique values in the unnamed column
# if they match, we can assume that it's the review_id
print(len(df['Unnamed: 0'].unique()), len(df))

9614 9614


# Most frequent values analysis
This could find default or placeholder values

In [7]:
# iterate over the columns and print the most frequent values
for label, content in df.items():
    print(label)
    print(content.mode())
    print()

Unnamed: 0
0           1
1           2
2           3
3           4
4           5
        ...  
9609    11993
9610    11994
9611    11995
9612    11998
9613    11999
Length: 9614, dtype: int64

product_id
0    B00FAPF5U0
dtype: object

product_parent
0    600633062
dtype: int64

product_title
0    Gone Girl
dtype: object

vine
0    N
dtype: object

verified_purchase
0    Y
dtype: object

review_headline
0    Five Stars
dtype: object

review_body
0    Great
dtype: object

review_date
0    2015-01-02
dtype: object

marketplace_id
0    0
dtype: int64

product_category_id
0    3
dtype: int64

label
0    False
dtype: bool



# Unique values analysis
This could find broken values in columns with just a couple of unique values.

In [8]:
# iterate over the columns and get the unique values
for label, content in df.items():
    print(label)
    print(content.unique())
    print()

Unnamed: 0
[    9    11    19 ... 11951 11953 11998]

product_id
['B001N2MZT8' 'B00GCBVE0Q' '1423165691' ... 'B000026FJ6' 'B00K6S2HS4'
 'B00B1COIRC']

product_parent
[903886718 282740618 883799517 ... 475533906  96406179 716679008]

product_title
['Green Zone [DVD]' 'Le secret de Green Knowe' 'A Disney Sketchbook.' ...
 'The Songs of Distant Earth' 'Tolles Album' 'Populaire']

vine
['N' 'Y']

verified_purchase
['Y' 'N']

review_headline
['green zone' nan 'okay mais...' ... 'Native (Gold Edition)'
 'Coup de coeur pour ce film adorable et original' 'Dans la lignée du 1er']

review_body
["I found at first it was a little difficult to associate Matt Damon as anything else but Jason Bourne.But as the film settled in I forgot Jason.The story wasn't overely bright but was conclusive as to what it was all about.That is the Americans covering a story which has already been told.i.e The Big Cover Up,of the Iraq affair.Overall I thought it was a good film , forget the politics watch the film.Good

# Check the rows with NaN values
The unique values analysis showed that the review_headline could be NaN, so we check the whole dataframe on NaN values. It seems that 715 rows don't have a review headline.

In [9]:
# show the rows with null in the 
df[df['review_headline'].isnull()]

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,NaN,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
7,49,B00005JYAX,581643936,Memento,N,N,NaN,"Le film est excellent niveau suspense,... Je l...",2002-09-28,0,3,True
35,383,B000024611,769213394,Phil Collins...Hits,N,Y,NaN,Un peu trop mou ce ne sont pas les versions le...,2014-08-29,2,3,False
62,642,B000050FII,676028219,Fight Club [Édition Collector],N,N,NaN,"Un film comme l'on en voit désormais très peu,...",2000-12-02,2,3,False
78,912,B009CAMJWQ,779021042,Jason Bourne : L'Héritage [Edition Limitée - B...,N,N,NaN,Un peu déçu par cet épisode qui n'est pas à la...,2013-08-14,2,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1084,11277,B005FGMBY2,558593095,Les Frères Grimm [Blu-ray],N,Y,NaN,Selon les gouts me direz-vous... C'est vrai et...,2014-12-11,2,3,False
1122,11631,B00AFBJIVQ,178375143,Les cinq légendes,N,Y,NaN,J'étais plutot retissante d'aller voir ce film...,2013-05-11,2,3,True
1129,11706,B00FAC2GKA,383229535,Brain Salad Surgery,N,Y,NaN,Je suis extrêmement déçu.. Cette édition conti...,2014-07-25,2,6,True
1131,11791,B0058HDHAK,766029498,One (Remasterisé),N,Y,NaN,C'est une compilation incontournable et qui pe...,2014-02-08,2,6,False


# Check which columns have NaN values
Other columns could also contain NaN values, so this is checked below. This way we know which columns we might have to fix.

In [10]:
# iterate over the columns and check if any row has NaN for this column
for label, content in df.items():
    print(label)
    print(content.isnull().any())
    print()

Unnamed: 0
False

product_id
False

product_parent
False

product_title
True

vine
False

verified_purchase
False

review_headline
True

review_body
False

review_date
True

marketplace_id
False

product_category_id
False

label
False



# Group the data on the product_id column
We expect the products to have the same product title, product parent and product category_id

In [11]:
# group the data on the product id 
df_gb = df.groupby('product_id')

## Get the products with multiple product titles

In [12]:
# get the counts and number of unique values per product_id group
aggs = df_gb.agg(['nunique']).reset_index(drop=False)

# print all rows with more than one unique product title
multiple_titles = aggs[aggs['product_title','nunique'] > 1]['product_id']

# most common titles of product ids with multiple titles 
titles= [[x, df_gb.get_group(x)['product_title'].mode()] for x in multiple_titles]
titles

[['0007288387',
  0            I Heart New York (I Heart Series)
  1    I Heart New York (I Heart Series, Book 1)
  dtype: object],
 ['0061438294',
  0    What Every BODY is Saying: An Ex-FBI Agent&#82...
  1    What Every BODY is Saying: An Ex-FBI Agent&#82...
  dtype: object],
 ['0099579936',
  0    Fifty Shades of Grey: Book One of the Fifty Sh...
  dtype: object],
 ['0224098101',
  0    Bridget Jones: Mad About the Boy
  1               Bridget est de retour
  dtype: object],
 ['0345529057',
  0    Bien mieux que la série, qui est pourtant géniale
  1    George R. R. Martin's A Game of Thrones 4-Book...
  dtype: object],
 ['0345803485',
  0    Fifty Shades of Grey: Book One of the Fifty Sh...
  1    Fifty Shades of Grey: Book One of the Fifty Sh...
  dtype: object],
 ['0375836675',
  0    I Am the Messenger
  1    I am the Messenger
  dtype: object],
 ['0385484518',
  0    Tuesdays with Morrie: An Old Man, A Young Man ...
  dtype: object],
 ['0590353403',
  0    Harry Potter and th

### Interesting data:
I think that the spacing in front of the titles is a weird print thing.

```
 0                                              Awesome
 1    Harry Potter, volume 1: Harry Potter and the P...
 dtype: object,
 
 0    American IV : The Man Comes Around
 1    American IV:  The Man Comes Around
 dtype: object,
 
 0    Sticks  + Stones
 1     Sticks + Stones
 dtype: object,
 
 0           Obscured By Clouds (remastered)
 1    Obscured By Clouds [Discovery Edition]
 dtype: object,
 
 0    Doctor Who - Complete Series 1-4 Box Set [23 D...
 1            Doctor Who: Series 1 - 4 Collection [DVD]
 dtype: object,
 
 0     Pink - Funhouse Tour/Live in Australia [Blu-ray]
 1    Pink - Funhouse Tour: Live In Australia [Blu-ray]
 dtype: object,
 
 0    Good Evening New York City [2 CD + 1 DVD]
 1      Good evening New York City (2CD + 1DVD)
 dtype: object,
 
 0                     Amazing
 1    Spánnénd bis zúm Schlúss
 2            Zwei von Drei...
 dtype: object,
 
 0     FarmVille 2: Country Escape
 1    FarmVille 2: Raus auf's Land
 dtype: object,
```


In [13]:
# checking the rows of a product
df[df['product_id'] == 'B0024RID60']

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
723,7116,B0024RID60,849107619,Some Girls,N,N,Die letzte große Platte der Stones,Ich kann mich noch sehr gut an die damaligen P...,2011-10-23,0,6,True
458,4659,B0024RID60,849107619,Remarquable !!!,N,N,Some Girls,"Attention, chef d'oeuvre : Some Girls, sorti e...",2011-02-08,2,6,True


# Get the rows with the switched product_title and review_headline

In [14]:
# most common titles of product ids with multiple titles 
unique_titles = [[x, df_gb.get_group(x)['product_title'].unique()] for x in multiple_titles]
unique_titles

[['0007288387',
  array(['I Heart New York (I Heart Series)',
         'I Heart New York (I Heart Series, Book 1)'], dtype=object)],
 ['0061438294',
  array(['What Every BODY is Saying: An Ex-FBI Agent&#8217;s Guide to Speed-Reading People',
         "What Every BODY is Saying: An Ex-FBI Agent&#8217;s Guide to Speed-Reading People: An Ex-FBI Agent's Guide to Speed-reading People"],
        dtype=object)],
 ['0099579936',
  array(['Fifty Shades of Grey',
         'Fifty Shades of Grey: Book One of the Fifty Shades Trilogy (Fifty Shades of Grey Series)'],
        dtype=object)],
 ['0224098101',
  array(['Bridget Jones: Mad About the Boy', 'Bridget est de retour'],
        dtype=object)],
 ['0345529057',
  array(["George R. R. Martin's A Game of Thrones 4-Book Boxed Set: A Game of Thrones, A Clash of Kings, A Storm of Swords, and A Feast for Crows (A Song of Ice and Fire)",
         'Bien mieux que la série, qui est pourtant géniale'], dtype=object)],
 ['0345803485',
  array(['Fifty Shade

In [15]:
def fix_title_headline(df, row):
    
    # get the possible titles of the product id of this row
    rows = df[df['product_id'] == row['product_id']]
    
    # the unique titles of this product
    unique_titles = rows['product_title'].unique()
    
    # get the review headline
    review_headline = row['review_headline']
    
    # check if the review_headline is in the unique titles
    if review_headline in unique_titles:
        
        # replace the review headline of the row with the title
        row['review_headline'] = row['product_title']
        
        # replace the title of the row with the review headline
        row['product_title'] = review_headline
        
    # return the modified row
    return row

In [16]:
df.apply(lambda x: fix_title_headline(df, x), axis=1)

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
0,9,B001N2MZT8,903886718,Green Zone [DVD],N,Y,green zone,I found at first it was a little difficult to ...,2010-11-15,1,3,False
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,NaN,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
2,19,1423165691,883799517,A Disney Sketchbook.,N,N,okay mais...,est-ce une coincidence que la plupart des prin...,2012-12-22,0,0,False
3,33,0061091480,623343977,Your Erroneous Zones,N,N,Arrogant,Wáyné Dyér is á pớpúlár áméricán pérsớnál grớw...,2009-07-21,0,0,True
4,34,B00HZ4CYOY,647510225,König der Mathematik Junior,N,Y,Tớllé Máthé Ápp...,.....unsere Kids mögen diese Art des Lernens. ...,2015-06-01,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1147,11940,B0011V9XT6,475533906,Beyoncé: The Beyonce Experience - Live [Blu-ra...,N,N,Blu-ray definitely all regions,I don't know why Amazon and one reviewer here ...,2009-12-29,1,3,False
1148,11944,B000026FJ6,96406179,The Songs of Distant Earth,N,Y,Wohlklingende Entspannung,"Ich mag diese Musik sehr, es trifft genau mein...",2013-12-18,0,6,True
1149,11951,B00K6S2HS4,409414789,Tolles Album,N,Y,Native (Gold Edition),"Tolle Band, tolles Album, tolle Lieder, einfac...",2015-02-23,0,6,False
1150,11953,B00B1COIRC,716679008,Populaire,N,N,Coup de coeur pour ce film adorable et original,R&eacute;jouissante com&eacute;die grâce &agra...,2015-04-10,0,3,True


## Get the products with multiple product parents

In [17]:
# print all rows with more than one unique product parent
multiple_parents = aggs[aggs['product_parent','nunique'] > 1]['product_id']

# most common parents of product ids with multiple parents 
parents = [df_gb.get_group(x)['product_parent'].mode() for x in multiple_parents]
parents

[]

## Get the products with multiple product categories
0 means it's a book, not that it's missing.

In [18]:
# print all rows with more than one unique product category
multiple_categories = aggs[aggs['product_category_id','nunique'] > 1]['product_id']

# print unique category ids of product ids with multiple categories 
categories = [df_gb.get_group(x)['product_category_id'].unique() for x in multiple_categories]
categories

[array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([ 0, 24], dtype=int64),
 array([ 0, 24], dtype=int64),
 array([1, 0], dtype=int64),
 array([ 0, 24], dtype=int64),
 array([ 0, 24], dtype=int64),
 array([24,  0], dtype=int64),
 array([24,  0], dtype=int64),
 array([0, 1], dtype=int64),
 array([ 0, 24], dtype=int64),
 array([0, 1], dtype=int64),
 array([1, 0], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([ 0, 15], dtype=int64),
 array([0, 1], dtype=int64),
 array([1, 0], dtype=int64),
 array([ 0, 24], dtype=int64),
 array([ 0, 24], dtype=int64),
 array([0, 1], dtype=int64),
 array([ 0, 24], dtype=int64),
 array([ 0, 15], dtype=int64),
 array([0, 1], dtype=int64),
 array([ 0, 15], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([ 0, 15], dtype=int64),
 array([ 0, 15], dtype=int64),
 array([ 0, 15], dtype=int64),
 array([ 0, 15], dtype=int64),
 array([ 0, 24], dtype=int64),
 array(

# Check the marketplace languages
I saw some review bodies with [escaped characters](https://fantasai.tripod.com/FHQR/Characode/EscCodes.htm), all normal vowels with an accent aigu, random &-signs and html line breaks. The languages of marketplace 1, 2 and 3 are correct. Marketplace 0 contains all 3 languages. We might want to try to predict the language so that we could assign them to a marketplace.

In [19]:
# show the review headline and body of a certain markteplace id 
# 0 = null/no particular marketplace 
# 1 = english 
# 2 = french 
# 3 = german
df[df['marketplace_id'] == 3][['review_headline','review_body']]

,review_headline,review_body
33,Ein Klassiker,Ein Muss für jede Plattensammlung. Geht richti...
117,"alles drin, was man braucht","Für den unschlagbaren Preis von 12,97 Euro ein..."
153,Slash,Dá méiné Vớrrédnér dié Détáils béléúchtét hábé...
304,Star Trek Into Darkness (Blu-ray 3D + DVD + Di...,Spannend wie der erste Star Trek 11. Action ge...
384,Minútés tớ Midnight-Dié Kúnst: Linkin Párks sc...,Da in den vergangenen Monaten viele Fake-Alben...
...,...,...
938,Placebo Loud like Love,Ich bin total begeistert von dieser CD. Man ka...
969,Gut!,Guter Thriller in dem man Tom Cruise sogar den...
1027,außergewöhnlich in Stimmer und Lied,Loreen ist einfach Spitzenklasse. Ich höre sie...
1066,Éinfách núr Súpér.,"Bis auf ein oder zwei Stücke, die arg schnulzi..."


# Check if there are invalid date values

In [20]:
# funtion that tries to make a datetime from the string
# true means valid, false means invalid, None means NaN
def vali_date(date_text):
    try:
        if not isinstance(date_text, str) and np.isnan(date_text): return None
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        return False

In [21]:
# get the valid dates (True, False or None) and count them
vali_dates = [vali_date(x) for x in df['review_date']]
Counter(vali_dates)

Counter({True: 9613, None: 1})

# Get a good threshold for the accent spam in the review body
It seems like 0.11 is a good threshold

In [22]:
def non_ascii_ratio(review_body):
    
    review_body = str(review_body)
    
    # the total number of characters
    total = len(review_body)
    
    # the number of accents
    asciis = 0
    
    # iterate over the characters and check if acute is in their unicode name
    for char in review_body:
        
        if len(char.encode()) > 1:
            asciis += 1
            
    # return the ratio of non ascii characters
    return asciis/total

In [23]:
# copy the dataframe, add the non ascii ratio and show the rows from a certain threshold (it shows that)
df_copy = df.copy()
df_copy['non_ascii_ratio'] = df_copy['review_body'].apply(non_ascii_ratio)
df_copy = df_copy.sort_values('non_ascii_ratio')
df_copy[df_copy['non_ascii_ratio'] > 0.07].head()

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label,non_ascii_ratio
897,8809,B00AFBJJJW,178375143,Les Cinq légendes [Combo Blu-ray 3D + Blu-ray ...,N,Y,NaN,Un vrai chef d’œuvre visuel. L'histoire est tr...,2013-04-11,0,3,False,0.075862
1187,11618,B009IA9OLE,278434220,Devil's Attorney,N,Y,NaN,"Très bon, il fait passer le temps mais très tr...",2014-07-05,2,1,False,0.088235
348,3706,B00QW8TYWO,828652896,Crossy Road,N,Y,AMAZING,When I first played this game it was AMAZING a...,2015-02-05,1,1,False,0.109677
734,7226,B00GU40HWQ,430605778,Sớớớớớớ Cớớớớớớlllllll !!!!!!!!!!!,N,Y,Criminal Minds - Die komplette achte Staffel [...,Ist dié Géilsté Sérié dér Wélt !!!!! !!!!!! !!...,2014-02-16,0,3,True,0.119048
712,7121,B007SVESDO,992326428,Marvel's The Avengers,N,N,hmhmmmmm....,"... ich vérstéhé nicht, wárúm diésér film núr ...",2012-06-02,0,3,True,0.146939


In [24]:
# check the review body of the row around the possible threshold that doesn't have a clear accent spam
df[df['Unnamed: 0'] == 3706]['review_body'].values[0]

'When I first played this game it was AMAZING and now no have nearly collected all the characters my favourite is the flea and the universe😜😜😜😝😝😝😍😍😍😎😘😘😘😻😻😸😸'

In [25]:
import unidecode
accented = df[df['Unnamed: 0'] == 7226]['review_body'].values[0]
print(accented)
unaccented = unidecode.unidecode(accented)
print(unaccented)

Ist dié Géilsté Sérié dér Wélt !!!!! !!!!!! !!!!! &#60;3 &#60;3  Liébé Grüßé  Jénnifér Kérn &#60;3 &#60;3 &#60;3 &#60;3 &#60;3
Ist die Geilste Serie der Welt !!!!! !!!!!! !!!!! &#60;3 &#60;3  Liebe Grusse  Jennifer Kern &#60;3 &#60;3 &#60;3 &#60;3 &#60;3


# Get a good threshold for the accent spam in the review headline
It seems like 0.11 is a good threshold for any language but French, we might want to keep that in mind

In [26]:
# copy the dataframe, add the non ascii ratio and show the rows from a certain threshold (it shows that)
df_copy = df.copy()
df_copy['non_ascii_ratio'] = df_copy['review_headline'].apply(non_ascii_ratio)
df_copy = df_copy.sort_values('non_ascii_ratio')
df_copy[(df_copy['non_ascii_ratio'] > 0.11) & (df_copy['marketplace_id'] != 2)].head()

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label,non_ascii_ratio
949,9317,B006OOHYIC,618772576,Quell Reflect,N,Y,Kniffélig,Éin knifféligés únd éntspánnéndés Spiél mit Sú...,2013-07-14,0,1,False,0.111111
832,8669,B005C3HBHU,167297708,Jeeves and Wooster - The Complete Collection [...,N,Y,Göttlich~,Álléiné dié érsté Épisớdé wár jédén Cént wért....,2014-04-26,0,3,True,0.111111
1053,10416,B000071IY2,982206081,Star Trek 10 - Nemesis,N,Y,DVD lässt zu wünschen übrig,"Die einen wollen Action, die anderen Philosoph...",2003-07-28,0,11,True,0.111111
476,4906,B00G6SDX36,140219026,Spigen Ultra Fit Coque pour iPhone 5/5S Transp...,N,Y,Très bonne qualité,"La Coque est de très bonne qualité, elle est f...",2015-08-11,0,17,False,0.111111
599,5889,B000025W7O,824346667,Toto IV,N,N,Ihr größter Erfolg,Im April 1982 erschien das vierte Album der U....,2015-05-29,0,6,True,0.111111
